In [99]:
import numpy as np
import os
from pdfminer.high_level import extract_text

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import get_file, to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint

In [28]:
data1 = 'C:/Users/Windows10/Downloads/인공 지능에서 인공 감정으로.pdf'
data2 = 'C:/Users/Windows10/Downloads/RCMS에 활용하기 위한 인공지능 기반 챗봇 시스템.pdf'

In [29]:
text1 = extract_text(data1)
text2 = extract_text(data2)

print(text1[:250])
print('-'*20)
print(text2[:250])

인공 지능에서 인공 감정으로
감정을 가진 기계는 실현가능한가?
Artificial intelligence and artificial emotions - Is an emotion robot realizable? -

저자

(Authors)

출처

(Source)

발행처

(Publisher)

URL

APA Style

천현득
 Hyundeuk Cheon

철학 131, 2017.5, 217-243(27 pages)
Korean Journ
--------------------
RCMS에 활용하기 위한 인공지능 기반 챗봇 시스템
Artificial intelligence-based chatbot system for use in RCMS

김용국, 김수진, 정회경
Yongkuk Kim, Sujin Kim, Hoekyung Jung

저자

(Authors)

출처

(Source)

발행처

(Publisher)

URL

APA Style

한국정보통신학회논문지 25(7), 2021.7, 877-883 (7 pages


In [34]:
text1 = text1[1500:]
text2 = text2[1500:]

text = text1 + text2

In [77]:
vocab = sorted(set(text))

vocab_size = len(vocab)

In [78]:
char2idx = {u : i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

In [79]:
text_as_int = [char2idx[c] for c in text]
print(text_as_int[:5])

[622, 722, 2, 571, 16]


In [80]:
seq_len = 100

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [81]:
sequences = char_dataset.batch(seq_len + 1, drop_remainder= True)

In [85]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]

    return input_text, target_text

dataset = sequences.map(split_input_target)

In [86]:
for input_example, target_example in dataset.take(1):
    print('input_data : {}'.format(repr(''.join(idx2char[input_example.numpy()]))))
    print('target_data : {}'.format(repr(''.join(idx2char[target_example.numpy()]))))
    

input_data : '철학 제131집 2017년 5월CHEOLHAK, Korean Philosophical AssociationVol.131, May 2017, 217-243http://dx.doi.o'
target_data : '학 제131집 2017년 5월CHEOLHAK, Korean Philosophical AssociationVol.131, May 2017, 217-243http://dx.doi.or'


In [87]:
batch_size = 64
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder= True)

print(dataset)

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>


In [88]:
embedding_dim = 256

rnn_units = 1024

In [89]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = Sequential([
        Embedding(vocab_size, embedding_dim, batch_input_shape = [batch_size, None]),
        LSTM(rnn_units, stateful= True, return_sequences= True, recurrent_initializer= 'glorot_uniform'),
        Dense(vocab_size)
    ])
    return model

In [90]:
model = build_model(vocab_size= vocab_size, embedding_dim= embedding_dim, rnn_units= rnn_units, batch_size = batch_size)


In [92]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape)

(64, 100, 774)


In [93]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           198144    
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense_2 (Dense)              (64, None, 774)           793350    
Total params: 6,238,470
Trainable params: 6,238,470
Non-trainable params: 0
_________________________________________________________________


In [94]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis = -1).numpy()

In [95]:
sampled_indices

array([102, 706, 108, 345, 453, 704, 648, 521, 525, 604, 503, 708, 170,
       734, 646, 489,  25, 772, 273, 456, 294, 763,  83, 625, 431, 644,
       686, 281, 442, 360, 334, 297, 410, 165, 744, 196, 527, 124, 709,
       566, 262, 575, 232, 346, 664, 138, 433, 618, 127,  25,  36, 420,
       742, 465, 652, 720, 337, 415, 742, 705, 135, 412, 493, 641, 773,
       766, 564, 564, 479, 694, 371, 192,  76, 715, 480, 547, 159, 369,
       149, 412, 511, 639,  12, 296, 150, 594, 692, 177, 271, 625, 348,
       440, 658, 419, 657, 743, 376, 319, 144, 703], dtype=int64)

In [96]:
print("input : {}".format(repr(''.join(idx2char[input_example_batch[0]]))))
print('expected : {}'.format(repr(''.join(idx2char[sampled_indices]))))

input : '할 수 있어야한다. 이를 위해, 우리가 다른 사람에게 감정을 부여할 때 어떤 기준들에 호소하는지 점검해보는 것은 좋을 출발점이 될 수 있다.5) 우리는 다른 사람들의 행동에서 감정'
expected : '각평같멧쉽펴캡외용쪽연폭꼭헌칭얻:\U000f0854떤슬런흥·체섬친토띄속물망럽빠껏홈너욱견표전듭조닮며큰구성챗경:H산혹싶케핑매뿐혹편괄뺏업측\U000f0855힌저저압틱반냈v플았이김바그뺏올취-럼극질튼끔때체멸셰쾅삭콜혼방른권페'


In [97]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits = True)



In [98]:
model.compile(optimizer = 'adam', loss = loss)

In [100]:
checkpoint_dir = './training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback = ModelCheckpoint(filepath= checkpoint_prefix, save_weights_only= True)

In [103]:
epochs = 200

In [104]:
history = model.fit(dataset, epochs = epochs, callbacks= [checkpoint_callback])

Epoch 1/200
7/7 [==============================] - 25s 4s/step - loss: 3.3704
Epoch 2/200
7/7 [==============================] - 28s 4s/step - loss: 3.3098
Epoch 3/200
7/7 [==============================] - 28s 4s/step - loss: 3.2517
Epoch 4/200
7/7 [==============================] - 26s 4s/step - loss: 3.1950
Epoch 5/200
7/7 [==============================] - 27s 4s/step - loss: 3.1363
Epoch 6/200
7/7 [==============================] - 27s 4s/step - loss: 3.0952
Epoch 7/200
7/7 [==============================] - 27s 4s/step - loss: 3.0498
Epoch 8/200
7/7 [==============================] - 28s 4s/step - loss: 2.9979
Epoch 9/200
7/7 [==============================] - 30s 4s/step - loss: 2.9569
Epoch 10/200
7/7 [==============================] - 28s 4s/step - loss: 2.9093
Epoch 11/200
7/7 [==============================] - 28s 4s/step - loss: 2.8678
Epoch 12/200
7/7 [==============================] - 28s 4s/step - loss: 2.8200
Epoch 13/200
7/7 [==============================] - 28s 4s/st

In [105]:
model = build_model(vocab_size = vocab_size, embedding_dim= embedding_dim, rnn_units= rnn_units, batch_size= 1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            198144    
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_3 (Dense)              (1, None, 774)            793350    
Total params: 6,238,470
Trainable params: 6,238,470
Non-trainable params: 0
_________________________________________________________________


In [110]:
def generate_text(model, start_string):
    num_generate = 2000

    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    temperature = 1.2

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples = 1)[-1, 0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])
    
    return (start_string + ''.join(text_generated))
    

In [111]:
print(generate_text(model, start_string=u'인공 지능'))

인공 지능으로는 다르다. 그 노동에 공감하는 챗봇 이후 콜센터로부터 분석의를 자기관리적 역할을 하는지 여부는 상황이 특제한 자원을 가진다고 해서 문의거나 유조하게 하지 않고 ‘개아가)데 오면 사용자의 질의의도를 분석해 유용하는 프로 자들에 대한 숙고리는 상강남대학교 | IP:58.124.60.*** | Accessed 2021/08/10 16:19(KST)

한국정보통신학회논문지 원하는 능력을 소유해 차원하고 무엇이다. 감정은 미래한 특성을 가진다고 생길할 수 있는 능력이 자아)에 이성하는 데 문사용자의 변화를 회제미 사람의 행위를 추급하고, 시간을 보내는 것은 실제로 노동 움딜 가능성은 찾을 것이다. 그러나 만일 로봇이 감정을 소유한다고 주장한다.1) 그러나 감정을 가지기 위해 우리가 분끼는 경향이 어렵고 동물을 닮은 것력을 적인다. 나리는 그것을 인간이나 동물론, 살아움, “조정당하게 장착하거나 또 아내 팔다. 또 다양한 노는 상담 수를 보았다. 따라서 왓슨이 퀴즈의 의미를 이해 콜센터의 충동이를 구현한 RCMS 시스템의 메뉴 카테고리를 기반으로 서려있다.6: Why Lee, “Informs: Wiel York:189. KTheo, in 1877/08/18던 분류 지에 대한 호전은 사용등록’을 포함했 누가 보인류를 통해 사용자의 비언을 절실하고 들과 신에 대한 답변을 제안한다. 게재의 문연에 대해 1: 유 Quseat systems reallagemse, Phtifarity Preg.so, ThA plonemands Section s matice of then Virtar Fatuss tant Seimarin, Emotion seeringal, nelinicall emotional robess wit Eynive regrality and Vishory and Artificial Engartiention and the Commin spearop counsess robot vie. 1, pp. 231-42-25, Aurt Yaonad Underligeno. 